In [68]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from sklearn.model_selection import cross_val_score,KFold


import tensorflow as tf
import datetime
from tensorflow.keras.callbacks import TensorBoard

In [22]:
data=pd.read_csv('train_data_ready.csv')
x=data.iloc[:,7:22]
y=data.loc[:,'target']

In [117]:
model=Sequential()
model.add(Dense(5,activation='relu',input_dim=15))
model.add(Dense(15,activation='relu'))

model.add(Dense(1,activation='relu'))

model.compile(loss='binary_crossentropy',
               optimizer=SGD(lr=0.01, momentum=0.9), #RMSprop(learning_rate=0.001)
               metrics=['accuracy'])

C:\Users\wojjj\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [118]:
kfold=KFold(n_splits=5, shuffle=True,random_state=10)
scores=[]

In [119]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [120]:
for train_index, test_index in kfold.split(x):

    x_train,x_test=x.iloc[train_index],x.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]

    model.fit(x_train,y_train,batch_size=8,epochs=1000,verbose=0) #,callbacks=[tensorboard_callback] #<- jak sie doda do modelu to pokazuje statystyki ale wolno się trenuje 

    score=model.evaluate(x_test,y_test,verbose=1)
    scores.append(score[1])
    print(score)
print('Accuracy: %.2f%% (+/- %.2f%%)' % (np.mean(scores)*100, np.std(scores)*100))

1/1 [==============================] - 0s 172ms/step - loss: 11.5687 - accuracy: 0.2500
[11.568711280822754, 0.25]
1/1 [==============================] - 0s 29ms/step - loss: 11.5687 - accuracy: 0.2500
[11.568711280822754, 0.25]
1/1 [==============================] - 0s 30ms/step - loss: 12.0717 - accuracy: 0.2174
[12.071699142456055, 0.21739129722118378]
1/1 [==============================] - 0s 30ms/step - loss: 9.3891 - accuracy: 0.3913
[9.38909912109375, 0.3913043439388275]
1/1 [==============================] - 0s 33ms/step - loss: 11.4010 - accuracy: 0.2609
[11.40104866027832, 0.260869562625885]
Accuracy: 27.39% (+/- 6.05%)


In [121]:
model.summary()
# %tensorboard --logdir logs/fit

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 5)                 80        
                                                                 
 dense_86 (Dense)            (None, 15)                90        
                                                                 
 dense_87 (Dense)            (None, 1)                 16        
                                                                 
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.save('modelV1.h5')